In [1]:
import cv2
import math
import random
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2
import random
import joblib
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
import functools, operator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.inception_v3 import InceptionV3

# **All Functions**

In [2]:
def pad_frames(frames, limit, jpegs=False):
        last_frame = frames[-1]
        if jpegs:
            frames_padded = frames + [last_frame]*(limit-len(frames))
        else:
            padding = np.asarray([last_frame * 1.]*(limit-len(frames)))
            frames_padded = np.concatenate([frames, padding], axis=0)
        return frames_padded

def resize_frames(frames):
  new_frames = []
  for frame in frames:
    new_frame = cv2.resize(frame,(224,224))
    new_frames.append(new_frame)
  
  return new_frames

def extract_frames_sampled(frames, K):
        n_frames = len(frames)
        splits = [i  for i in range(0,n_frames,14)]
#         print(splits)
        sub_frames = []

        for idx in splits:
          sub_frames.append(frames[idx])
        return sub_frames

    
def calc_histscore(frames):
    hist_score = dict()
    for i in range(0,len(frames)):
        for j in range (i+1,len(frames)):
            hist_img1 = cv2.calcHist([frames[i]], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
            cv2.normalize(hist_img1, hist_img1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
            hist_img2 = cv2.calcHist([frames[j]], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
            cv2.normalize(hist_img2, hist_img2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
            metric_val = cv2.compareHist(hist_img1, hist_img2, cv2.HISTCMP_CHISQR)
            hist_score[(i,j)] = metric_val
    return hist_score
    



def fitness_func(frames,delta,fit_idx,hist_score ):
#     print('HIST_SCORE : ',hist_score)
    hist_data = []
    score1 = 0
    score2 = []  #importance score
    final_score = 0
#     print('LEN = ',len(frames))

    for i in range(0,len(frames)):
        for j in range (i+1,len(frames)):
            metric_val = hist_score[(fit_idx[i],fit_idx[j])]
            hist_data.append(metric_val)
            
    mean = np.mean(hist_data)
    std = np.std(hist_data)
            
    for i in range(0,len(frames)):
        net_mean = 0
        c = 0
        for j in range (i+1,len(frames)):
            metric_val = hist_score[(fit_idx[i],fit_idx[j])]
#             print('Metric',metric_val)
            if metric_val < (mean+std):
                net_mean +=metric_val
                c+=1
        if c>0:
           net_mean = net_mean/(c)
        if mean>0:
           net_mean/=mean
        score2.append(math.log(delta[i])*(math.log(1/(net_mean+1))))
#         print('SCORE 2 : ',score2)
    
#     print('NET_MEAN : ',net_mean)
    
    for i in range(0,len(frames)):
        net_mean = 0
        c = 0
        for j in range (i+1,len(frames)):
            metric_val  = hist_score[(i,j)]
            final_score += (metric_val*score2[i]*score2[j])/((i-j)*(i-j))
            
    return final_score


def crossover(l, q):
#     print(len(l),len(q))
# generating the random number to perform crossover
    k = random.randint(0, len(l))
    print("Crossover point :", k)
# interchanging the genes
    for i in range(k, len(q)):
        l[i], q[i] = q[i], l[i]
    return l, q


def mutation(l):
    count0 = []
    count1 = []
    for i in range(len(l)):
        if(l[i]=='0'):
            count0.append(i)
        else:
            count1.append(i)
            
    mutation_points = []
    change = ''
    req = min(len(l),28)
    if len(count1) <req:
        mutation_points = random.sample(count0,req-len(count1))
        change = '1'
    elif len(count1) >req:
        mutation_points = random.sample(count1,len(count1)-req)
        change = '0'
        
    for i in mutation_points:
        l[i] = change
    
    return l
#     print(mutation_points)
def select_parents(population,fitness_values):
  parents = []
  total = sum(fitness_values)
  if total == 0:
    return population
  norm_fitness_values = [x/total for x in fitness_values]

  #find cumulative fitness values for roulette wheel selection
  cumulative_fitness = []
  start = 0
  for norm_value in norm_fitness_values:
    start+=norm_value
    cumulative_fitness.append(start)

  population_size = len(population)
  for count in range(population_size):
    random_number = random.uniform(0, 1)
    individual_number = 0
    for score in cumulative_fitness:
      if(random_number<=score):
        parents.append(population[individual_number])
        break
      individual_number+=1
      
  return parents

In [3]:
import os
dir_path = r'/kaggle/input/msvd-clips/YouTubeClips/'
res = []
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)

In [4]:
def get_frames(frames,solution):
    final_frames = []
    for i in range(len(frames)):
        if(solution[i]=='1'):
           final_frames.append(frames[i])
    return final_frames

#Googlenet model
def get_model():
    i = tf.keras.layers.Input([224, 224, 3], dtype = tf.float32)
    x = tf.keras.applications.inception_v3.preprocess_input(i)
    model = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3),pooling='avg')
    out = model(x)
    model=tf.keras.models.Model(inputs=[i], outputs=out)
    return model
#   model.summary()
model = get_model()

2023-01-20 17:21:18.765344: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


87924736/87910968 [==============================] - 5s 0us/step


In [5]:
K = 28

for file in res:
        print(file)
        cap = cv2.VideoCapture(dir_path + str(file))
#         cap = cv2.VideoCapture('/kaggle/input/msvd-clips/YouTubeClips/-_hbPLsZvvo_288_305.avi')

        frames = []
        while  cap.isOpened():
            ret, frame = cap.read()
            if not ret: break # break if no next frame
            frames.append(frame)
#         print(frames)

        if len(frames) < K:
                frames = pad_frames(frames,K,True)
        
        print(len(frames))
        frames = resize_frames(frames)
      
        frames = extract_frames_sampled(frames,14)
#         frames = resize_frames(frames)
#         print('Frames : ',len(frames))
        print(frames[0].size)
        
        metric_values = []
        for i in range(1,len(frames)):
        #         hist_img1 = cv2.calcHist(frames[i], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
                hist_img1 = cv2.calcHist([frames[i]], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
                cv2.normalize(hist_img1, hist_img1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        #         hist_img2 = cv2.calcHist(frames[i-1], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
                hist_img2 = cv2.calcHist([frames[i-1]], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
                cv2.normalize(hist_img2, hist_img2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
                metric_val1 = cv2.compareHist(hist_img1, hist_img2, cv2.HISTCMP_CHISQR)
                metric_values.append(metric_val1)
                
        delta = []
        
        mean = np.mean(metric_values)
#         print('Mean =' + str(mean))
        processed_frames = []
    
        for i in range(0,len(metric_values)):
            if metric_values[i] >= mean : 
                processed_frames.append(frames[i])
                delta.append(i+1)
                
        print('Processed Frames Length ',len(processed_frames))
#         print(processed_frames)
        hist_score = dict()

            
        hist_score = calc_histscore(processed_frames)
#         print('HIST',hist_score)
        
        txt = ""
        chromosome = txt.zfill(len(processed_frames))
        # print(len(chromosome))
        # print(chromosome)

        p1 = list(chromosome)
        p2 = list(chromosome)
        p3 = list(chromosome)
        p4 = list(chromosome)

        c1 = min(28,len(processed_frames))

        for i in range(c1):
            p1[i]='1'
            p2[i]='1'
            p3[i]='1'
            p4[i]='1'

        #generating random population
        random.shuffle(p1)
        random.shuffle(p2)
        random.shuffle(p3)
        random.shuffle(p4)


#         print(p1)
#         print(p2)
#         print(p3)
#         print(p4)
        
        
        iteration = 10
        threshold = 300
        solution = []

#         for i in range(iteration):
#             print('ITERATION : ',i)
#             p1,p2 = crossover(p1,p2)
#             p1 = mutation(p1)
#             p2 = mutation(p2)

#             fitness1 = [processed_frames[i] for i in range(len(p1)) if p1[i] == '1']
#             fitness2 = [processed_frames[i] for i in range(len(p2)) if p2[i] == '1']

#             fit1_idx = [i for i in range(len(p1)) if p1[i] == '1']
#             fit2_idx = [i for i in range(len(p2)) if p2[i] == '1']

#             fit1_score = fitness_func(fitness1,delta,fit1_idx,hist_score )
#             fit2_score = fitness_func(fitness2,delta,fit2_idx,hist_score )
            
#             print('Score ',abs(fit1_score-fit2_score),fit1_score,fit2_score)

#             if(abs(fit1_score-fit2_score)<threshold or i == iteration):
#                 solution = fitness1
#                 break
                
#         print(len(solution))
#         if len(solution) <28:
#             solution = pad_frames(solution,28)
        iteration = 6#10 
        threshold = 300
        solution = []

        prev_generation = [p1,p2,p3,p4]
        curr_generation = []
        max_fitness_idx = 0

        for i in range(iteration):
#                 print('ITERATION : ',i)
#                 print(prev_generation[0])
#                 print('---------------------------------------------------')
                p1,p2,p3,p4 = prev_generation[0],prev_generation[1],prev_generation[2],prev_generation[3]
                fitness1 = [processed_frames[j] for j in range(len(p1)) if p1[j] == '1']
                fitness2 = [processed_frames[j] for j in range(len(p2)) if p2[j] == '1']
                fitness3 = [processed_frames[j] for j in range(len(p3)) if p3[j] == '1']
                fitness4 = [processed_frames[j] for j in range(len(p4)) if p4[j] == '1']


                fit1_idx = [j for j in range(len(p1)) if p1[j] == '1']
                fit2_idx = [j for j in range(len(p2)) if p2[j] == '1']
                fit3_idx = [j for j in range(len(p3)) if p3[j] == '1']
                fit4_idx = [j for j in range(len(p4)) if p4[j] == '1']

            #     print(fit1_idx)
            #     print(fit2_idx)

                fit1_score = fitness_func(fitness1,delta,fit1_idx,hist_score )
                fit2_score = fitness_func(fitness2,delta,fit2_idx,hist_score )
                fit3_score = fitness_func(fitness3,delta,fit3_idx,hist_score )
                fit4_score = fitness_func(fitness4,delta,fit4_idx,hist_score )

#                 print('NetFitnessScore : ',sum([fit1_score,fit2_score,fit3_score,fit4_score]))
                while len(curr_generation)<4:
                    parent1,parent2,parent3,parent4 = select_parents(prev_generation,[fit1_score,fit2_score,fit3_score,fit4_score])

            #     print(fit1_score)
                    parent1,parent2 = crossover(parent1,parent2)
                    parent1 = mutation(parent1)
                    parent2 = mutation(parent2)

                    curr_generation.append(parent1)
                    curr_generation.append(parent2)

                prev_generation = curr_generation
                curr_generation = []
#                 print('---------------------------------------------------')

        #     print('NET SCORE : ',abs(fit1_score-fit2_score))

        # if(abs(fit1_score-fit2_score)<threshold or i+1 == iteration):
        solution = prev_generation[0]
        print(solution)
        solution = get_frames(processed_frames,solution)
        solution = resize_frames(solution)
        if(len(solution)<28):
            solution =  pad_frames(solution,28)
        solution = np.array(solution)
        
 
        solution=model.predict(solution,batch_size=128)
#         solution=pad_frames(solution,28)
        print('Size of a frame : ',solution[0].size,'Total Frames : ',len(solution))
        
        np.save(file[:-4],solution)
#         print('-----------------------------------------------------------------------------------------------------------------')
#         print()

yREFkmrrYiw_51_57.avi
90
150528
Processed Frames Length  3
Crossover point : 2
Crossover point : 1
Crossover point : 0
Crossover point : 2
Crossover point : 2
Crossover point : 3
Crossover point : 0
Crossover point : 1
Crossover point : 1
Crossover point : 3
Crossover point : 0
Crossover point : 1
['1', '1', '1']


2023-01-20 17:21:29.550309: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Size of a frame :  2048 Total Frames :  28
ao-9B8IV9_E_217_221.avi
101
150528
Processed Frames Length  3
Crossover point : 3
Crossover point : 2
Crossover point : 2
Crossover point : 3
Crossover point : 0
Crossover point : 1
Crossover point : 2
Crossover point : 0
Crossover point : 1
Crossover point : 3
Crossover point : 2
Crossover point : 1
['1', '1', '1']
Size of a frame :  2048 Total Frames :  28
6ZhKOK08tgM_45_49.avi
120
150528
Processed Frames Length  4
Crossover point : 1
Crossover point : 3
Crossover point : 2
Crossover point : 3
Crossover point : 0
Crossover point : 2
Crossover point : 1
Crossover point : 0
Crossover point : 2
Crossover point : 0
Crossover point : 4
Crossover point : 4
['1', '1', '1', '1']
Size of a frame :  2048 Total Frames :  28
jxdubZzQrio_33_46.avi
391
150528
Processed Frames Length  5
Crossover point : 2
Crossover point : 1
Crossover point : 0
Crossover point : 5
Crossover point : 3
Crossover point : 4
Crossover point : 3
Crossover point : 2
Crossover po

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Size of a frame :  2048 Total Frames :  28
GY5Dl00LrEI_2_13.avi
276
150528
Processed Frames Length  7
Crossover point : 6
Crossover point : 5
Crossover point : 2
Crossover point : 7
Crossover point : 5
Crossover point : 2
Crossover point : 0
Crossover point : 7
Crossover point : 2
Crossover point : 6
Crossover point : 0
Crossover point : 1
['1', '1', '1', '1', '1', '1', '1']
Size of a frame :  2048 Total Frames :  28
AoNnzlGhI04_112_128.avi
481
150528
Processed Frames Length  9
Crossover point : 2
Crossover point : 4
Crossover point : 8
Crossover point : 5
Crossover point : 0
Crossover point : 8
Crossover point : 2
Crossover point : 5
Crossover point : 2
Crossover point : 8
Crossover point : 3
Crossover point : 0
['1', '1', '1', '1', '1', '1', '1', '1', '1']
Size of a frame :  2048 Total Frames :  28
HkpUWzNNVt4_20_30.avi
300
150528
Processed Frames Length  9
Crossover point : 0
Crossover point : 0
Crossover point : 0
Crossover point : 3
Crossover point : 2
Crossover point : 5
Crossove

In [6]:
# np.load('/kaggle/working/yREFkmrrYiw_51_57.npy')
# a = np.load('/kaggle/input/extracted-features/global_features/global_features/-wa0umYJVGg_23_41.npy')
# print(a)

In [7]:
# a = np.load('/kaggle/working/yREFkmrrYiw_51_57.npy')
# print(a)
